In [1]:
import os
os.chdir('../')
%pwd

'd:\\Programming\\aiTeaching'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir: Path
    contests_dir: Path
    authors_file: Path 

In [3]:
from src.aiteacher.constants import *
from src.aiteacher.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            contests_dir=config.contests_dir,
            authors_file=config.authors_file
        )
        return data_ingestion_config

In [47]:
import os
import urllib.request as request
import zipfile
from src.aiteacher import logger
import subprocess
import pandas as pd
import bz2

In [48]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.unzip_dir):
            create_directories([self.config.unzip_dir])
            logger.info(f"Cloning dataset from {self.config.source_url}")
            subprocess.run(["git", "clone", self.config.source_url, self.config.unzip_dir], check=True)
            logger.info("Dataset sucessfully cloned.")
        else:
            logger.info(f"Dataset already exists at {self.config.unzip_dir}")

    def decompress_bz2(self, file_path, output_path):
        """Decompress a .bz2 file to a CSV file."""
        with bz2.BZ2File(file_path, "rb") as f_in:
            with open(output_path, "wb") as f_out:
                f_out.write(f_in.read())

    def extract_data(self):
        try:
            contests_path = self.config.contests_dir
            file_paths = [f for f in os.listdir(contests_path) if f.endswith(".csv.bz2")]

            all_contests = []
            temp_dir = os.path.join(contests_path, "temp_extracted")
            os.makedirs(temp_dir, exist_ok=True)

            for file in file_paths:
                contest_id = file.split(".")[0]
                compressed_file_path = os.path.join(contests_path, file)
                extracted_file_path = os.path.join(temp_dir, f"{contest_id}.csv")
                
                # Decompress the file
                self.decompress_bz2(compressed_file_path, extracted_file_path)
                
                # Read the extracted CSV
                df = pd.read_csv(extracted_file_path, engine="python", on_bad_lines="skip")
                df["contest_id"] = contest_id
                all_contests.append(df)
                logger.info(f"Loaded {len(df)} records from contest {contest_id}")

            if all_contests:
                save_path = self.config.local_data_file
                create_directories([os.path.dirname(save_path)])
                final_df = pd.concat(all_contests, ignore_index=True)
                final_df.to_csv(save_path, index=False)
                logger.info(f"All contests saved to {save_path}.")
                return save_path
        except Exception as e:
            logger.error(f"Error extracting data: {e}")
            raise

In [49]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

# data_ingestion.download_file()

[2025-03-14 04:00:55,368: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-14 04:00:55,369: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-14 04:00:55,369: INFO: common: created directioy at: artifacts]
[2025-03-14 04:00:55,369: INFO: common: created directioy at: artifacts/data_ingestion]


In [50]:
data_ingestion.extract_data()

[2025-03-14 04:00:59,612: INFO: 1095698680: Loaded 107 records from contest 1]


[2025-03-14 04:00:59,757: INFO: 1095698680: Loaded 939 records from contest 10]
[2025-03-14 04:00:59,804: INFO: 1095698680: Loaded 891 records from contest 100]
[2025-03-14 04:01:00,192: INFO: 1095698680: Loaded 6139 records from contest 1000]
[2025-03-14 04:01:00,430: INFO: 1095698680: Loaded 2602 records from contest 1001]
[2025-03-14 04:01:00,808: INFO: 1095698680: Loaded 3466 records from contest 1002]
[2025-03-14 04:01:01,261: INFO: 1095698680: Loaded 8016 records from contest 1003]
[2025-03-14 04:01:01,693: INFO: 1095698680: Loaded 8084 records from contest 1004]
[2025-03-14 04:01:02,105: INFO: 1095698680: Loaded 8766 records from contest 1005]
[2025-03-14 04:01:02,584: INFO: 1095698680: Loaded 9805 records from contest 1006]
[2025-03-14 04:01:02,672: INFO: 1095698680: Loaded 662 records from contest 1007]
[2025-03-14 04:01:03,309: INFO: 1095698680: Loaded 10360 records from contest 1008]
[2025-03-14 04:01:03,795: INFO: 1095698680: Loaded 9083 records from contest 1009]
[2025-03-

C:\Users\siddk\AppData\Local\Temp\ipykernel_18796\1095698680.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_contests, ignore_index=True)


[2025-03-14 04:40:06,491: ERROR: 1095698680: Error extracting data: [Errno 28] No space left on device]


OSError: [Errno 28] No space left on device